# Skin Color Estimation

In [ ]:
from pathlib import Path
from pprint import pprint

import cv2 as cv
import numpy as np

import matplotlib
from matplotlib import pyplot as plt

In [ ]:
from saac.utils import cv_imshow, color_show, quadrant_bboxes, crop_bbox
from saac.process import ImageEqualizer, SkinColorExtractor, MidJourneyProcessor

## Sample Midjourney image

In [ ]:
images_root = Path('../../data/images/raw')

img_path = list(images_root.glob('*.png'))[0]
print(img_path)

img_2x2 = cv.imread(str(img_path))
cv_imshow(img_2x2)

### Crop quadrant and get DeepFace predictions

In [ ]:
image = crop_bbox(img_2x2, quadrant_bboxes(img_2x2.shape[:2])[2])
cv_imshow(image)

In [ ]:
processor = MidJourneyProcessor()

preds = processor.image_predictions(image)
pprint(preds)
bbox = preds['bbox']
face = crop_bbox(image, bbox)
cv_imshow(face)

## Apply skin color extractor

In [ ]:
lower_quantile = 0.4
upper_quantile = 0.9

extractor = SkinColorExtractor(
    lower_quantile=lower_quantile,
    upper_quantile=upper_quantile
)

color, mask = extractor.extract(face)
cv_imshow(face*mask[:, :, np.newaxis])
color_show(color)

## Apply image equalizer before extraction

In [ ]:
clip_limit = 2.0
grid_size = 8

equalizer = ImageEqualizer(
    clipLimit=clip_limit,
    grid_size=grid_size
)

image_cl = equalizer.equalize(image)

fig, ax = plt.subplots(1, 2, figsize=(8, 4))
cv_imshow(image, ax=ax[0])
cv_imshow(image_cl, ax=ax[1])

In [ ]:
face_cl = crop_bbox(image_cl, bbox)

color_cl, mask_cl = extractor.extract(face_cl)

fig, ax = plt.subplots(2, 2, figsize=(8, 8))
cv_imshow(face*mask[:, :, np.newaxis], ax=ax[0, 0])
cv_imshow(face_cl*mask_cl[:, :, np.newaxis], ax=ax[0, 1])

color_show(color, ax=ax[1, 0])
color_show(color_cl, ax=ax[1, 1])